In [38]:
import os
from dotenv import load_dotenv
load_dotenv()
import pandas as pd
import boto3
import csv
import xml.etree.ElementTree as ET

In [39]:


# List of XML file paths
xml_file_paths = ['resources/grobid/xml/Level1_combined.grobid.tei.xml', 'resources/grobid/xml/Level2_combined.grobid.tei.xml','resources/grobid/xml/Level3_combined.grobid.tei.xml']

# Create a CSV file
csv_filename = 'resources/grobid/outcomes.csv'

with open(csv_filename, 'w', newline='', encoding='utf-8') as csvfile:
    csv_writer = csv.writer(csvfile)
    
    # Write header
    csv_writer.writerow(['Topic', 'Learning Outcomes Section'])
    
    # Iterate through each XML file
    for xml_file_path in xml_file_paths:
        print(xml_file_path)
        # Read the XML data from the file
        with open(xml_file_path, 'r') as file:
            xml_data = file.read()
        # Parse the XML data
        root = ET.fromstring(xml_data)

        for div_element in root.findall('.//{http://www.tei-c.org/ns/1.0}div'):
                head_element = div_element.find('.//{http://www.tei-c.org/ns/1.0}head')

                p_elements = div_element.findall('.//{http://www.tei-c.org/ns/1.0}p')

                combined_p_text = ' '.join(p_element.text for p_element in p_elements if p_element.text)
                if combined_p_text != '':
                    csv_writer.writerow([head_element.text if head_element is not None else '', combined_p_text])

print(f"Combined CSV file '{csv_filename}' created successfully.")



resources/grobid/xml/Level1_combined.grobid.tei.xml
resources/grobid/xml/Level2_combined.grobid.tei.xml
resources/grobid/xml/Level3_combined.grobid.tei.xml
Combined CSV file 'resources/grobid/outcomes.csv' created successfully.


In [40]:
def remove_extra_whitespaces(value):
    if isinstance(value, str):
        return ' '.join(value.split())
    else:
        return value

def preprocess_text(value):
    if isinstance(value, str):
        return f'"{value}"'
    else:
        return value

In [41]:

aws_access_key_id = os.getenv('aws_access_key_id')
aws_secret_access_key = os.getenv('aws_secret_access_key')
bucket_name = 'cfainstitute-learning-outcomes-raw'

df = pd.read_csv('resources/grobid/outcomes.csv', dtype=str)
df = df.map(remove_extra_whitespaces)
df = df.map(preprocess_text)

df.to_csv('resources/grobid/outcomes_processed.csv', index=False)

s3 = boto3.client('s3', aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key)

csv_data = df.to_csv(index=False)

s3_key = 'outcomes_processed.csv'

s3.put_object(Body=csv_data, Bucket=bucket_name, Key=s3_key)

print(f"CSV file has been uploaded to S3 at s3://{bucket_name}/{s3_key}")


CSV file has been uploaded to S3 at s3://cfainstitute-learning-outcomes-raw/outcomes_processed.csv


# Upload to snowflake from local

In [42]:
from sqlalchemy import create_engine

snowflake_account = os.getenv('snowflake_account')
snowflake_user = os.getenv('snowflake_user')
snowflake_password = os.getenv('snowflake_password')
snowflake_warehouse = os.getenv('snowflake_warehouse')
snowflake_schema = os.getenv('snowflake_schema')
snowflake_database = 'your_database'

engine = create_engine(
    'snowflake://{user}:{password}@{account_identifier}/'.format(
        user=snowflake_user,
        password=snowflake_password,
        account_identifier=snowflake_account,
    )
)
connection = engine.connect()


## Create table and db

In [43]:
target_database = 'CFAInstitute'
target_table = 'Topic_Learning_Outcomes'

create_database_query = f"CREATE DATABASE IF NOT EXISTS {target_database}"
connection.execute(create_database_query)

use_database_query = f"USE DATABASE {target_database}"
connection.execute(use_database_query)
connection.execute("USE WAREHOUSE TEST")

create_table_query = f"""
CREATE TABLE IF NOT EXISTS {target_table} (
    Topic VARCHAR,
    "Learning Outcomes Section" VARCHAR
    
)
"""

connection.execute(create_table_query)


In [44]:
connection.execute(f"TRUNCATE TABLE {target_table}")

connection.execute(f"PUT file://resources/grobid/outcomes_processed.csv @%{target_table}")

connection.execute(f"COPY INTO {target_table} ON_ERROR=CONTINUE FILE_FORMAT = (FIELD_OPTIONALLY_ENCLOSED_BY = '\"' SKIP_HEADER=1 PARSE_HEADER = FALSE)")


# upload to snowflake from s3

In [45]:
connection.execute("""CREATE OR REPLACE FILE FORMAT mycsvformat
   TYPE = 'CSV'
   FIELD_DELIMITER = '|'
   SKIP_HEADER = 1;""")

 Goto iam of aws and create a role
 use above role arn to create storage integration
 Create STORAGE INTEGRATION that can connect to aws account

In [46]:
# cursor.execute("""CREATE OR REPLACE STORAGE INTEGRATION s3_int2
#   TYPE = EXTERNAL_STAGE
#   STORAGE_PROVIDER = 'S3'
#   STORAGE_AWS_ROLE_ARN = 'arn:aws:iam::640055273174:role/s3-read'
#   ENABLED = TRUE
#   STORAGE_ALLOWED_LOCATIONS = ('*')
# """)

 go to trusted relationships of iam role created in above step from aws console and change it to below template
```
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "Service": "s3.amazonaws.com",
                "AWS": "<user arn>"
            },
            "Action": "sts:AssumeRole",
            "Condition": {
                "StringEquals": {
                    "sts:ExternalId": "<external id>"
                }
            }
        }
    ]
}
```
run "DESC Integration s3_int2"  inside snowflake to get STORAGE_AWS_IAM_USER_ARN and STORAGE_AWS_EXTERNAL_ID of storage integration crreated

In [47]:
# connection.execute("""CREATE OR REPLACE STAGE external_stage
#   FILE_FORMAT = mycsvformat
#   URL = 's3://cfainstitute-learning-outcomes-raw/outcomes.csv'
#   STORAGE_INTEGRATION = s3_int2;
# """)

In [48]:
# Upload to snowflake from s3

In [51]:
connection.execute("""
copy into Topic_Learning_Outcomes
from 's3://cfainstitute-learning-outcomes-raw/outcomes_processed.csv'
storage_integration = s3_int2
FORCE = TRUE
ON_ERROR = CONTINUE
  file_format = (type = csv FIELD_OPTIONALLY_ENCLOSED_BY = '"' SKIP_HEADER=1 PARSE_HEADER = FALSE);
""")